In [5]:
from ecommerce.operations.base.read import ReadOperation
from ecommerce.operations.custom.feature_extractor import FeatureExtractionOperation
from ecommerce.operations.custom.data_splitter import SplitDataOperation
from ecommerce.configs import columns as c

In [6]:
# data = ReadCriteoSearchData(path="/Users/maciej.filanowicz/ecommerce/data/raw/CriteoSearchData",object_format='tsv', header=0).execute()
# WriteOperation(data=data, 
#                path="/Users/maciej.filanowicz/ecommerce/data/standardized/df_source.parquet",
#                data_format="parquet", 
#                mode="w", 
#                partition_cols=[c.PARTNER_ID]).execute()

In [7]:
data = ReadOperation(path="/Users/maciej.filanowicz/ecommerce/data/standardized/df_source.parquet/partner_id=E68029E9BCE099A60571AF757CBB6A08",object_format='parquet').execute()
df_features = FeatureExtractionOperation().execute(df=data)
df_train, df_validation, df_test = SplitDataOperation().execute(df=df_features, validation_days=7, test_days=7)

/Users/maciej.filanowicz/ecommerce/ecommerce/operations/custom/feature_extractor.py:115: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with string, please explicitly cast to a compatible dtype first.
  df.loc[:, c.SALES_AMOUNT_IN_EURO] = df[c.SALES_AMOUNT_IN_EURO].astype(float).apply(lambda x: max(x, 0))
/Users/maciej.filanowicz/ecommerce/ecommerce/operations/custom/feature_extractor.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with string, please explicitly cast to a compatible dtype first.
  df.loc[:, c.PRODUCT_PRICE] = df[c.PRODUCT_PRICE].apply(float)
